In [25]:
## Packages need for data pre-process
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

from scipy import sparse
from collections import Counter

import nltk
from numpy import savetxt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

import itertools

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# For evaluztion
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

# Running time
from timeit import default_timer as timer

# Visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure

In [21]:
# Import preprocessed Dataset
four_after = pd.read_csv('/home/rep/scRNA-seq_clustering_to_Twitter/P1_preprocessing/doc_metadata_stemming_four_users.csv')
del four_after['Unnamed: 0']
True_Label = list(four_after["user_id_new"])

In [22]:
four_after

,user_id,user_id_new,screen_name,text,tweets_processed
0,27902825,2,UMichFootball,Leave it all on the field! @UMichFootball! Bes...,leav field umichfootbal best rivalri colleg fo...
1,27902825,2,UMichFootball,There’s no time to look backwards… only ahead!...,time look ahead watch umichfootbal ball goblu ...
2,27902825,2,UMichFootball,It’s called “The Game’ for a reason. \r\n\r\n#...,call game reason goblu beatosu
3,27902825,2,UMichFootball,"On Saturday, our seniors will play their final...",saturday senior play final game big hous senio...
4,27902825,2,UMichFootball,The Glasgow Decade with @UMichFootball is just...,glasgow decad umichfootbal complet stori goblu...
...,...,...,...,...,...
12502,19071682,3,breakingweather,A flash flood emergency is in effect for south...,flash flood emerg effect southwestern arkansa ...
12503,19071682,3,breakingweather,"Now that Barry, the first hurricane to make U....",barri first hurrican make landfal year come go...
12504,19071682,3,breakingweather,"Showers and locally heavy, drenching thunderst...",shower local heavi drench thunderstorm associ ...
12505,19071682,3,breakingweather,While Monday felt like a typical summer day in...,monday felt like typic summer day northeast te...


In [23]:
# read in Monocle3 clustering results 
Monocle3 = pd.read_csv("M3_files/Monocle3_estimated_clustering_four_PCA_25PCs.csv")
Monocle3_results = pd.DataFrame(Monocle3.drop(["Unnamed: 0", "Index"], axis =1))
col_name = Monocle3_results.columns.to_list()

#### (1) Purity

In [24]:
# Purity
Correct_target = pd.DataFrame(four_after[["screen_name", "user_id_new"]])
Monocle3_purity = []
for i, j in enumerate(col_name):
    n = pd.DataFrame(Monocle3_results[j])
    df_compare = pd.concat([n, Correct_target], axis=1)
    numerator = df_compare.groupby([j, "user_id_new"], as_index=False)['screen_name'].count().sort_values('screen_name', ascending=False).drop_duplicates(j)["screen_name"].sum()
    purity = numerator/len(True_Label)
    Monocle3_purity.append(purity)
    
Monocle3_purity

[0.9542656112576957,
 0.9525865515311426,
 0.9525865515311426,
 0.9525865515311426,
 0.951707044055329,
 0.951707044055329,
 0.951707044055329,
 0.951707044055329,
 0.951786999280403,
 0.951786999280403,
 0.9505876709042936,
 0.9485088350523707,
 0.951786999280403,
 0.951786999280403,
 0.951786999280403,
 0.9449908051491165,
 0.9430718797473415,
 0.9418725513712322,
 0.9386743423682737,
 0.9386743423682737,
 0.9386743423682737,
 0.9386743423682737,
 0.9386743423682737,
 0.9311585512113216,
 0.9288398496841769,
 0.9288398496841769,
 0.7312704885264252,
 0.7312704885264252,
 0.7312704885264252]

#### (2) AMI

In [27]:
Monocle3_AMI = []
for i, j in enumerate(col_name):
    n = Monocle3_results[j]
    AMI = adjusted_mutual_info_score(True_Label, n)
    Monocle3_AMI.append(AMI)
    
Monocle3_AMI

[0.4887389231457672,
 0.49923690131522097,
 0.5326384700729032,
 0.5561466366028073,
 0.5584263229018126,
 0.5824159765151695,
 0.599795977305118,
 0.6204959886465161,
 0.6551438776105953,
 0.647993781306791,
 0.6741188735014783,
 0.6524570217871397,
 0.6931405616555849,
 0.6757319805260859,
 0.712003711823654,
 0.712126318096543,
 0.7377290472186024,
 0.7456181387268351,
 0.7775071466356813,
 0.7864307556444614,
 0.7864307556444614,
 0.7864307556444614,
 0.7864307556444614,
 0.781471094732301,
 0.7825040469070079,
 0.7825040469070079,
 0.7115919333607148,
 0.7115919333607148,
 0.7115919333607148]

#### (3) ARI

In [26]:
Monocle3_ARI = []
for i, j in enumerate(col_name):
    n = Monocle3_results[j]
    ARI = adjusted_rand_score(True_Label, n)
    Monocle3_ARI.append(ARI)
    
Monocle3_ARI

[0.19048065938922967,
 0.20872024665614097,
 0.28763294014357066,
 0.3405864472824282,
 0.3402750299327325,
 0.41572833020376526,
 0.4685622252265951,
 0.5317899269733319,
 0.6129601734877436,
 0.6076614807007896,
 0.644333865738104,
 0.6097686182901669,
 0.6948226026034522,
 0.6404690145279871,
 0.722526648340752,
 0.739813234850148,
 0.7864909229342937,
 0.7939111829479063,
 0.8292474396362922,
 0.8392311609345792,
 0.8392311609345792,
 0.8392311609345792,
 0.8392311609345792,
 0.8306243609927774,
 0.8235349779347572,
 0.8235349779347572,
 0.6253573209464632,
 0.6253573209464632,
 0.6253573209464632]

#### Summary

In [29]:
Monocle3_evaluation_metrics = {'Purity': Monocle3_purity, 'AMI': Monocle3_AMI, 'ARI': Monocle3_ARI}
Monocle3_evaluation_metrics = pd.DataFrame(data = Monocle3_evaluation_metrics)
Monocle3_evaluation_metrics

,Purity,AMI,ARI
0,0.954266,0.488739,0.190481
1,0.952587,0.499237,0.208720
2,0.952587,0.532638,0.287633
3,0.952587,0.556147,0.340586
4,0.951707,0.558426,0.340275
5,0.951707,0.582416,0.415728
6,0.951707,0.599796,0.468562
7,0.951707,0.620496,0.531790
8,0.951787,0.655144,0.612960
9,0.951787,0.647994,0.607661


In [30]:
# Monocle3_evaluation_metrics.to_csv("M3_files/Monocle3_evaluation_metrics_four_PCA_PCs25.csv")